In [1]:
!nvidia-smi


Tue Oct 31 22:02:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.40       Driver Version: 516.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P3    12W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


In [3]:
! pip install --upgrade accelerate
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB 445.2 kB/s eta 0:00:13
     ---------------------------------------- 0.1/5.7 MB 550.5 kB/s eta 0:00:11
     ---------------------------------------- 0.1/5.7 MB 491.5 kB/s eta 0:00:12
      --------------------------------------- 0.1/5.7 MB 658.7 kB/s eta 0:00:09
     - -------------------------------------- 0.2/5.7 MB 657.6 kB/s eta 0:00:09
     - -------------------------------------- 0.2/5.7 MB 692.4 kB/s eta 0:00:08
     - -------------------------------------- 0.2/5.7 MB 655.6 kB/s eta 0:00:09
     - -------------------------------------- 0.2/5.7 MB 655.6 kB/s eta 0:00:09
     - -------------------------------------- 0.2/5.7 MB 535.1 kB/s eta 0:00:11
     - -------------------------------------- 0.3/5.7 MB 525.1 kB/s eta 0:00:11
     - -------------------------------------- 0.3/5.7 MB 5

In [3]:

from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\omkar\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [10]:
torch.cuda.is_available()

False

In [9]:
! pip install torch>=1.13.0+cu116 torchvision>=0.13.0+cu116 torchaudio>=0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116


In [5]:
%%capture
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)